In [1]:
import cv2
import dlib
import math

def get_eye_aspect_ratio(eye):
    A = math.sqrt((eye[1][0] - eye[5][0])**2 + (eye[1][1] - eye[5][1])**2)
    B = math.sqrt((eye[2][0] - eye[4][0])**2 + (eye[2][1] - eye[4][1])**2)
    C = math.sqrt((eye[0][0] - eye[3][0])**2 + (eye[0][1] - eye[3][1])**2)
    ear = (A + B) / (2.0 * C)
    return ear

def check_drowsiness(landmarks):
    left_eye = landmarks[36:42]
    right_eye = landmarks[42:48]
    left_ear = get_eye_aspect_ratio(left_eye)
    right_ear = get_eye_aspect_ratio(right_eye)
    avg_ear = (left_ear + right_ear) / 2.0
    threshold = 0.25
    return avg_ear < threshold

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        landmarks_list = [(p.x, p.y) for p in landmarks.parts()]
        drowsy = check_drowsiness(landmarks_list)

        # Get the face bounding box coordinates
        (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())

        # Draw a red box if drowsy, otherwise draw a green box
        color = (0, 0, 255) if drowsy else (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

        if drowsy:
            cv2.putText(frame, "Drowsy", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            cv2.putText(frame, "Awake", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
